In [3]:
import numpy as np
import numpy.random as nrand
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import itertools
from itertools import combinations
import copy

In [4]:
# Load raw data
df_all = pd.read_csv('trna_Domingo_data.csv')
seqs_raw = df_all.seq.to_numpy()
seqs_raw = np.array(list(map(list,seqs_raw))) # Convert to 2D array
seqs = seqs_raw[:,np.array([1,2,6,27,43,46,66,69,70,71])-1] # Keep variable sites

In [5]:
#Generate neighbor_list for N_max calculation

neighbor_list = []
for seq in seqs:
    neighbor_list.append(np.where((seqs != seq).sum(axis=1) == 1)[0])
# with open('../../index_file/trna_Domingo_neighbor_list.pkl','wb') as f:
#     pickle.dump(neighbor_list,f)

In [ ]:
# # Generate epi_square_list for epi calculation

epi_square_list = []
for idx_00, seq_00 in enumerate(seqs,start=0): 
    idx_11_list = np.where((seqs != seq_00).sum(axis=1) == 2)[0]
    idx_11_list_filtered = idx_11_list[idx_11_list>idx_00]
    for idx_11 in idx_11_list_filtered:
        seq_11 = seqs[idx_11]
        pos1,pos2 = np.where(seq_11 != seq_00)[0]
        mut1,mut2 = seq_11[seq_11 != seq_00]
        seq_10 = np.copy(seq_00); seq_01 = np.copy(seq_00)
        seq_10[pos1] = mut1
        seq_01[pos2] = mut2
        if len(np.where((seqs != seq_10).sum(axis=1) == 0)[0]) == 0 or \
           len(np.where((seqs != seq_01).sum(axis=1) == 0)[0]) == 0:
            continue
        idx_10 = np.where((seqs != seq_10).sum(axis=1) == 0)[0][0]
        idx_01 = np.where((seqs != seq_01).sum(axis=1) == 0)[0][0]
        if idx_10 is None or idx_01 is None:
            continue
        elif idx_10 < idx_01 and idx_10 > idx_00:
            epi_square_list.append([idx_00,idx_10,idx_01,idx_11])
        elif idx_10 > idx_01 and idx_01 > idx_00:
            epi_square_list.append([idx_00,idx_01,idx_10,idx_11])

epi_square_list = np.array(epi_square_list)
# with open('../../index_file/trna_Domingo_epi_square_list.pkl','wb') as f:
#     pickle.dump(epi_square_list,f)

In [ ]:
# Generate pathway_list for open_ratio calculation

seqs_list = [''.join(seq) for seq in seqs]
seqidx_dict = {seqs_list[i]:i for i in range(len(df_all))}
pathway_list = []
for idx_0, seq_0 in enumerate(seqs,start=0): 
    print(idx_0,end='\r')
    idx_4_list = np.where((seqs != seq_0).sum(axis=1) == 4)[0]
    idx_4_list_filtered = idx_4_list[idx_4_list>idx_0]
    for idx_4 in idx_4_list_filtered:
        seq_4 = seqs[idx_4]
        poss = np.where(seq_4 != seq_0)[0]
        muts = seq_4[seq_4 != seq_0]
        
        for order in itertools.permutations(range(4),3):
            pathway_sub_list = [idx_0]
            seq_tmp = np.copy(seq_0)
            for step in range(3):
                pos = poss[order[step]]; mut = muts[order[step]]
                seq_tmp[pos] = mut
                seq_tmp_string = ''.join(seq_tmp)
                if seq_tmp_string in seqidx_dict:
                    pathway_sub_list.append(seqidx_dict[seq_tmp_string])
                else:
                    break
                if step == 2:
                    pathway_sub_list.append(idx_4)
                    pathway_list.append(pathway_sub_list)

# with open('../../index_file/trna_Domingo_pathway_list_4steps_all.pkl','wb') as f:
#     pickle.dump(pathway_list,f)